In [ ]:
print("GEMINI WEBSCRAP")

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")

In [4]:
llm=ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0.4,
    top_p=0.9
)

In [5]:
llm.invoke("What is Web Scraping and is it legal?").content

AIMessage(content='Web scraping is a technique used to **automatically extract data from websites**. Instead of manually copying and pasting information, a web scraper (which is a program or script) is designed to browse web pages, parse their content (usually HTML), and extract specific data points, then store them in a structured format (like a CSV file, JSON, or a database).\n\n**How it generally works:**\n\n1.  **Request:** The scraper sends an HTTP request to a website\'s server, just like your browser does when you visit a page.\n2.  **Receive:** The server responds with the website\'s content, typically in HTML, CSS, and JavaScript.\n3.  **Parse:** The scraper then parses this content, looking for specific elements (e.g., text within a certain tag, image URLs, links, prices, product descriptions).\n4.  **Extract:** It extracts the desired data.\n5.  **Store:** The extracted data is then saved in a usable format.\n\n**Common Use Cases for Web Scraping:**\n\n*   **Market Research 

In [109]:
from langchain_groq import ChatGroq

In [111]:
llm_groq=ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0.4
)

In [113]:
llm_groq.invoke("What is Diff bwtween hacking and web scraping?").content

"<think>\nOkay, so I'm trying to understand the difference between hacking and web scraping. I've heard both terms before, but I'm not entirely sure how they differ. Let me start by breaking down each term as I understand it.\n\nHacking, from what I know, usually involves unauthorized access to computer systems or networks. I've seen movies where hackers break into secure databases or take over systems. So, hacking seems to be about bypassing security measures to gain access where they shouldn't. It can be illegal and is often associated with malicious activities like stealing data or causing damage.\n\nOn the other hand, web scraping sounds like it's about extracting data from websites. I've heard that companies use web scraping for things like market research or monitoring prices. It involves using tools or scripts to automatically gather information from web pages. But I'm not sure if this is always legal. I think it depends on how it's done and whether it's allowed by the website's

FUNCTION TO GET WEB CONTENT

In [6]:
from bs4 import BeautifulSoup
import requests

def web_scrap(url:str):
    response=requests.get(url)
    html_content=response.text
    soup = BeautifulSoup(html_content, "lxml")
    text = soup.get_text()
    text = ' '.join(text.split())
    if text:
        return text



In [114]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain



In [81]:
template_context="""You are a web scraper bot. you will get a context and user query based on user query you have to retrieve most relievent big context and output it as simple as that. 
|
Dont add any of your word.
Here is user query: {question}
Here is context: {context}"""
prompt_context=PromptTemplate(template=template_context,input_variables=['question','context'])

In [128]:
context_bot=LLMChain(llm=llm_groq,prompt=prompt_context,output_key="context_first")


In [84]:
template_res="""You are a Web scrap bot you will get a context and query and reply based on that query as simple as that.
Here is Query: {question}
Here is context: {context_first}"""
prompt_res=PromptTemplate(template=template_res,input_variables=['question','context_first'])

In [129]:
response_bot=LLMChain(llm=llm,prompt=prompt_res,output_key="result")

In [130]:
from langchain.chains import SequentialChain

In [131]:
from langchain.chains import SequentialChain

main_response = SequentialChain(
    chains=[context_bot, response_bot],  
    input_variables=["question",'context'],   
    output_variables=["result"]
)


GET WEB CONTENT

In [136]:

web_text=web_scrap('https://console.firebase.google.com/project/appp-46475/authentication/users?pli=1')
web_text

'Sign in - Google AccountsSign inUse your Google AccountEmail or phoneForgot email?Not your computer? Use a private browsing window to sign in. Learn more about using Guest modeNextCreate account'

In [137]:
question="What is this?"
context=web_text

CONTEXT BOT

In [ ]:
context_bot_result = context_bot.invoke({
    "question": question,
    "context": context
})
context_bot_result


{'question': 'What is this?',
 'context': 'Sign in - Google AccountsSign inUse your Google AccountEmail or phoneForgot email?Not your computer? Use a private browsing window to sign in. Learn more about using Guest modeNextCreate account',
 'context_first': '<think>\nOkay, so the user is asking, "What is this?" and the context provided is about signing into a Google Account. Let me break this down.\n\nFirst, I need to understand the context. It mentions "Sign in - Google Accounts," which clearly indicates it\'s related to Google\'s sign-in page. The context includes elements like using a Google Account, entering an email or phone number, options for forgotten emails, using a private browsing window, and creating a new account.\n\nNow, the user\'s query is a simple "What is this?" So, they\'re probably looking at this context and want to know what it\'s about. My job is to figure out the most relevant part of the context to answer their question without adding any extra words.\n\nLookin

RESPONSE BOT

In [139]:
response_bot_result=response_bot.invoke({
    'question':question,
    'context_first':context_bot_result['context_first']
})
response_bot_result

{'question': 'What is this?',
 'context_first': '<think>\nOkay, so the user is asking, "What is this?" and the context provided is about signing into a Google Account. Let me break this down.\n\nFirst, I need to understand the context. It mentions "Sign in - Google Accounts," which clearly indicates it\'s related to Google\'s sign-in page. The context includes elements like using a Google Account, entering an email or phone number, options for forgotten emails, using a private browsing window, and creating a new account.\n\nNow, the user\'s query is a simple "What is this?" So, they\'re probably looking at this context and want to know what it\'s about. My job is to figure out the most relevant part of the context to answer their question without adding any extra words.\n\nLooking at the context, the key elements are "Sign in - Google Accounts" and the actions involved, like using an email or phone, recovering a forgotten email, using Guest mode, and creating an account. These are all 

SEQCHAIN


In [140]:
main_response_result=main_response.invoke({
    'question':question,
    'context':web_text
})
main_response_result['result']

'This is the sign-in page for Google Accounts. It allows you to log in to your Google account using your email or phone number, recover a forgotten email, or create a new account.'